### Imports

In [242]:
import pandas as pd
import plotly.graph_objects as go

### Read data from 'xlsx' file

In [243]:
# Read 'xlsx' file as dataframe
df = pd.read_excel('../data/processed/data_cuba.xlsx', index_col=None, sheet_name='base_limpia')
# Create a list with dataframe column names
cols = df.columns

### Create pivot tables from dataframe

In [244]:
"""
Function to create
pivot tables from
columns 102-109 and
index 78, 80 and 81
"""
def create_pivot_table():
    dfs = []
    index = [78, 80, 81]
    index_names = ["Episiotomía", "Kristeller", "Dilatación_Manual"]
    count_cols = 102
    for i in range(8):
        for j in range(3):
            # Create a pivot table from df
            table = pd.pivot_table(df, values=cols[0], index=cols[index[j]],
                            columns=cols[count_cols], aggfunc='count')      
            # Convert index to columns and reset index
            table = table.rename_axis('Procedimiento').reset_index().rename_axis('', axis='columns')
            # Drop a column by index
            table.drop(table.columns[1], axis=1, inplace=True)
            # Rename column
            table = table.rename(columns={'Sí': f'Secuela{i+1}'})
            # Drop a row by index and reset index
            table = table.drop(table.index[0]).reset_index(drop=True)
            # Rename values based on a condition from a selected column
            table.loc[table['Procedimiento'] == 'Sí', 'Procedimiento'] = index_names[j]
            # Append table to list
            dfs.append(table)
        count_cols += 1
    return dfs

dfs = create_pivot_table()

e = []
for i in range(0, 24, 3):
    e.append(dfs[i])

k = []
for i in range(1, 24, 3):
    k.append(dfs[i])
    
d = []
for i in range(2, 24, 3):
    d.append(dfs[i])

### Merge dataframes

In [245]:
"""
Function to me[]ge dataframes
"""
def mergeDataframes(*args, **kwargs):
    dfs = list(args)
    for i in range(len(dfs)):
        if i <= 1:
            df = pd.merge(dfs[0], dfs[1], on='Procedimiento', how ='outer')
        else:
            df = pd.merge(df, dfs[i], on='Procedimiento', how ='outer')
    return df

episiotomia = mergeDataframes(e[0], e[1], e[2], e[3], e[4], e[5], e[6], e[7])
kristeller = mergeDataframes(k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7])
dilatacion = mergeDataframes(d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7])

### Concatenate dataframes

In [246]:
# Create a list with dataframes
procedimientos = [episiotomia, kristeller, dilatacion]

# Concatenate the dataframes
df_join = pd.concat(procedimientos, ignore_index=True)

df_join

,Procedimiento,Secuela1,Secuela2,Secuela3,Secuela4,Secuela5,Secuela6,Secuela7,Secuela8
0,Episiotomía,2,3,4,8,3,4,4,NaN
1,Kristeller,2,3,4,8,3,4,3,1.0
2,Dilatación_Manual,3,4,5,9,4,5,3,2.0


In [260]:
cols[81]

'De los siguientes procedimientos, marca todos los que se practicaron en tu parto: [Dilatación manual]'

### Visualize dataframe with stacked bar chart

In [269]:
list_secuelas = ['Parálisis cerebral', 
            'Discapacidad intelectual',
            'Convulsiones',
            'Retardo psicomotor',
            'Epilepsia',
            'Trastornos de comunicación',
            'Alteraciones sensoriales',
            'Muerte']

fig = go.Figure(data=[
    go.Bar(name='Episiotomía', x=list_secuelas, y=df_join.iloc[0, 1:9], marker_color='mediumpurple'),
        go.Bar(name='Kristeller', x=list_secuelas, y=df_join.iloc[1, 1:9], marker_color='teal'),
        go.Bar(name='Dilatación Manual', x=list_secuelas, y=df_join.iloc[2, 1:9], marker_color='mediumvioletred')
    ])
# Change the bar mode
fig.update_layout(
    barmode='stack',
    title='Secuelas más frecuentes en el bebé de acuerdo al procedimiento realizado', 
    xaxis_title='Tipos de secuelas', 
    yaxis_title='Encuestadas',
    )
fig.show()
# fig.write_html('../dataviz/secuelas_bebe.html')